<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [21]:
import pandas as pd
import tweepy

In [22]:
keys = pd.read_csv("C:/Bases_kl/Llaves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))

In [23]:
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [24]:
tweets = tweepy.Cursor(api.search,
                       q=['cambio climático -RT'],#Quitelos tweets retweetiados
                       lang='es',
                       geocode='4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)#numero de tweets a buscar


In [25]:
t = [tweet for tweet in tweets]

In [26]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)

In [27]:
#creacióndepandas con los tweets    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweets,date,city,user_location,id,link
0,No se debe discriminar una forma de consumo de...,2021-05-20 01:39:52,Bogotá,"Bogotá, D.C., Colombia",1395192519579906048,https://twitter.com/i/web/stuatus/139519251957...
1,@pedroferneyc @MartinSantosR Jajajajajaja si y...,2021-05-20 01:14:56,Bogotá,"Bogotá, D.C., Colombia",1395186242380259330,https://twitter.com/i/web/stuatus/139518624238...
2,¡Juntos podemos hacerle frente al cambio climá...,2021-05-20 00:30:00,Bogotá,"Bogotá, Colombia",1395174934784118789,https://twitter.com/i/web/stuatus/139517493478...
3,Hoy en dia se habla de cultivos y practicas pa...,2021-05-20 00:08:01,Bogotá,"Bogotá, D.C., Colombia",1395169402811830275,https://twitter.com/i/web/stuatus/139516940281...
4,#CátedraTSE | La escasez de agua puede ser un ...,2021-05-20 00:00:00,Bogotá,"Bogotá, D.C., Colombia",1395167386588831748,https://twitter.com/i/web/stuatus/139516738658...


In [28]:
data = data[['tweets', 'date']]
data.head()

,tweets,date
0,No se debe discriminar una forma de consumo de...,2021-05-20 01:39:52
1,@pedroferneyc @MartinSantosR Jajajajajaja si y...,2021-05-20 01:14:56
2,¡Juntos podemos hacerle frente al cambio climá...,2021-05-20 00:30:00
3,Hoy en dia se habla de cultivos y practicas pa...,2021-05-20 00:08:01
4,#CátedraTSE | La escasez de agua puede ser un ...,2021-05-20 00:00:00


In [29]:
#Quitar hashtasg,emoticones, links
import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions

In [30]:
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,texto_limpio,links,hashtags,mentions
0,No se debe discriminar una forma de consumo de...,2021-05-20 01:39:52,No se debe discriminar una forma de consumo de...,[https://t.co/wszcVzgg6j],[],[]
1,@pedroferneyc @MartinSantosR Jajajajajaja si y...,2021-05-20 01:14:56,"Jajajajajaja si y de la esclavitud, del camb...",[],[],"[@pedroferneyc, @MartinSantosR, @IvanDuque]"
2,¡Juntos podemos hacerle frente al cambio climá...,2021-05-20 00:30:00,¡Juntos podemos hacerle frente al cambio climá...,"[https://t.co/Y1QspjPY5d, https://t.co/BexRDEx...","[#GeneraciónVerde,]",[@RochesterSchool]
3,Hoy en dia se habla de cultivos y practicas pa...,2021-05-20 00:08:01,Hoy en dia se habla de cultivos y practicas pa...,[https://t.co/l4xd6sTlwp],[],[]
4,#CátedraTSE | La escasez de agua puede ser un ...,2021-05-20 00:00:00,| La escasez de agua puede ser un factor dete...,[https://t.co/PCJM3D4JPR],[#CátedraTSE],[]
...,...,...,...,...,...,...
95,Cuando se creen que el mundo gira alrededor de...,2021-05-17 05:38:23,Cuando se creen que el mundo gira alrededor de...,[],[],[]
96,"Escuelas Bioclimáticas, un proyecto regional q...",2021-05-17 05:02:42,"Escuelas Bioclimáticas, un proyecto regional q...",[https://t.co/GfOYOOwKtm],[],[]
97,"#MissUniverso no me gustan estos concursos, pe...",2021-05-17 03:02:27,"no me gustan estos concursos, pero el resulta...",[],"[#MissUniverso, #ParoNacionalIndefinido, #SOSC...",[]
98,✨cambio climático✨,2021-05-17 02:42:09,✨cambio climático✨,[],[],[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [31]:
import pandas as pd
import regex
import emoji

#Función:extrae los emoji
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text) #busca los carateres que omiencen con \X que posiblemente esun emoji
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list

In [32]:
#Muestra en una columna del pandas los emojis utilizados
data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head(100)

,tweets,date,texto_limpio,links,hashtags,mentions,emojis
0,No se debe discriminar una forma de consumo de...,2021-05-20 01:39:52,No se debe discriminar una forma de consumo de...,[https://t.co/wszcVzgg6j],[],[],[]
1,@pedroferneyc @MartinSantosR Jajajajajaja si y...,2021-05-20 01:14:56,"Jajajajajaja si y de la esclavitud, del camb...",[],[],"[@pedroferneyc, @MartinSantosR, @IvanDuque]",[]
2,¡Juntos podemos hacerle frente al cambio climá...,2021-05-20 00:30:00,¡Juntos podemos hacerle frente al cambio climá...,"[https://t.co/Y1QspjPY5d, https://t.co/BexRDEx...","[#GeneraciónVerde,]",[@RochesterSchool],[👉]
3,Hoy en dia se habla de cultivos y practicas pa...,2021-05-20 00:08:01,Hoy en dia se habla de cultivos y practicas pa...,[https://t.co/l4xd6sTlwp],[],[],[]
4,#CátedraTSE | La escasez de agua puede ser un ...,2021-05-20 00:00:00,| La escasez de agua puede ser un factor dete...,[https://t.co/PCJM3D4JPR],[#CátedraTSE],[],[]
...,...,...,...,...,...,...,...
95,Cuando se creen que el mundo gira alrededor de...,2021-05-17 05:38:23,Cuando se creen que el mundo gira alrededor de...,[],[],[],[]
96,"Escuelas Bioclimáticas, un proyecto regional q...",2021-05-17 05:02:42,"Escuelas Bioclimáticas, un proyecto regional q...",[https://t.co/GfOYOOwKtm],[],[],[]
97,"#MissUniverso no me gustan estos concursos, pe...",2021-05-17 03:02:27,"no me gustan estos concursos, pero el resulta...",[],"[#MissUniverso, #ParoNacionalIndefinido, #SOSC...",[],[]
98,✨cambio climático✨,2021-05-17 02:42:09,✨cambio climático✨,[],[],[],"[✨, ✨]"


In [33]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [34]:
from collections import Counter
temp_emojis= pd.DataFrame(list(zip(Counter(sum(data.emojis.values, [])).keys(),Counter(sum(data.emojis.values, [])).values())))
temp_emojis.columns=['emoji','cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)

In [35]:
trace = go.Bar(x=temp_emojis.emoji.values,
                   y=temp_emojis.cnt.values)

layout = go.Layout(title="Emojis más usados - Cambio Climático")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)